# Network Map Plots

In [54]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install

from geocluster import geoClusters
from geoUtils import convertMetersToLat, convertLatToMeters, convertMetersToLong, convertLongToMeters
from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters
from ioUtils import loadJoblib, saveFile, getFile
from fsUtils import mkDir, mkSubDir, setFile, setSubDir
from folium import PolyLine, CircleMarker, Circle, Marker, Icon, FeatureGroup, Map
from pandas import Series

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))


Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-30 06:47:57.413806


In [4]:
savedir = "/Users/tgadf/Downloads/network/"

# Folium Functions

In [2]:
colors = [ 'red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'lightgray', 'black' ]

def addData(m, dc, name, geovalues, cmap, radius=75, useBB=False, fill=False, useIcon=False):
    from timeUtils import clock, elapsed
    from modelio import saveJoblib, loadJoblib
    from folium import PolyLine, CircleMarker, Circle, Marker, Icon, FeatureGroup
    start,cmt = clock("Adding to folium")
    
    feature_group = FeatureGroup(name=name)

    for geo,weight in geovalues.items():
        color='gray'
        if isinstance(cmap, dict):
            for val,col in cmap.items():
                if weight.get(val) is not None:
                    color=col
                    break
        else:
            color=cmap
        if useBB:
            points = dc.getBB(geo, istuple=True)
            PolyLine(points, color=color, weight=len(weight), opacity=1, popup=",".join(weight.keys())).add_to(feature_group)
        elif useIcon:
            center = dc.getCenter(geo)
            Marker(center, icon=Icon(color='blue', icon_color='white', icon=col, angle=0, prefix='fa'), popup=",".join(weight.keys())).add_to(feature_group)
        else:
            if fill:
                center = dc.getCenter(geo)
                Circle(center, color=color, radius=radius, fill=True, fill_color=color, weight=len(weight)/2, opacity=1, popup=", ".join(weight.keys())).add_to(feature_group)
            else:
                center = dc.getCenter(geo)
                Circle(center, color=color, radius=radius, weight=len(weight)/2, opacity=1, popup=", ".join(weight.keys())).add_to(feature_group)
                
    
    feature_group.add_to(m)

    
def addDriverData(m, mapdata):
    from folium import Marker, FeatureGroup, Icon, PolyLine, Circle
    from math import tanh

    feature_group   = FeatureGroup(name="Driver Home")
    feature_group_1 = FeatureGroup(name="Driver Top 3")
    feature_group_2 = FeatureGroup(name="Driver Top 10")
    feature_group_3 = FeatureGroup(name="Driver All")
    feature_group_t1 = FeatureGroup(name="Driver Top 10 Trips")
    feature_group_t2 = FeatureGroup(name="Driver Top 25 Trips")
    feature_group_t3 = FeatureGroup(name="Driver All Trips")
    
    for vertexNum in mapdata['Vertices'].keys():
        vertexData = mapdata['Vertices'][vertexNum]
        popup = "R = {0}  ;  D = {1}  ;  Attrs = {2}".format(vertexNum, vertexData['Degree'], vertexData['Attrs'])
        try:
            center = vertexData['CoM']
            center = [float(center[0]), float(center[1])]
        except:
            continue
        try:
            radius = vertexData['Radius']
        except:
            radius = None
        try:
            quantiles = vertexData['Quantiles']
        except:
            quantiles = None
        if vertexNum == "Home":
            Marker(center, icon=Icon(color='darkpurple', icon_color='white', icon="home", angle=0, prefix='fa'), popup=popup).add_to(feature_group)
            if radius is not None and radius > 0:
                Circle(center, color='darkpurple', radius=radius, fill=True, fill_color='darkpurple', weight=2, opacity=1).add_to(feature_group)
            if quantiles is not None:
                Circle(center, color='black', radius=quantiles[-1], fill=False, weight=1, opacity=1).add_to(feature_group)
        else:
            if int(vertexNum) < 3:
                Marker(center, icon=Icon(color='blue', icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_1)
                if radius is not None and radius > 0:
                    Circle(center, color='blue', radius=radius, fill=True, fill_color='blue', weight=2, opacity=1).add_to(feature_group_1)
                if quantiles is not None:
                    Circle(center, color='black', radius=quantiles[-1], fill=False, weight=1, opacity=1).add_to(feature_group_1)
            elif int(vertexNum) < 10:
                Marker(center, icon=Icon(color='green', icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_2)
                if radius is not None and radius > 0:
                    Circle(center, color='green', radius=radius, fill=True, fill_color='green', weight=2, opacity=1).add_to(feature_group_2)
                if quantiles is not None:
                    Circle(center, color='black', radius=quantiles[-1], fill=False, weight=1, opacity=1).add_to(feature_group_2)
            else:
                Marker(center, icon=Icon(color='orange', icon_color='white', icon="car", angle=0, prefix='fa'), popup=popup).add_to(feature_group_3)
                if radius is not None and radius > 0:
                    Circle(center, color='orange', radius=radius, fill=True, fill_color='orange', weight=2, opacity=1).add_to(feature_group_3)
                if quantiles is not None:
                    Circle(center, color='orange', radius=quantiles[-1], fill=False, weight=1, opacity=1).add_to(feature_group_3)

    maxWeight = None
    for edgeNum in mapdata['Edges'].keys():
        edgeData = mapdata['Edges'][edgeNum]
        points   = edgeData["CoM"]
        dist     = edgeData["Dist"]
        dow      = edgeData["DOW"]
        if not all(isinstance(p, tuple) for p in points):
            continue
        weight   = edgeData['Weight']
        if maxWeight is None:
            maxWeight = weight
        lweight  = 1*tanh(2*weight/maxWeight)
        popup = "R = {0}  ;  Trips = {1}  ;  Dist = {2}  ;  Day-of-Week = {3}".format(edgeNum, weight, dist, dow)
        if int(edgeNum) < 10:
            color='darkblue'
            PolyLine(points, color=color, weight=lweight, opacity=1, popup=popup).add_to(feature_group_t1)
        elif int(edgeNum) < 25:
            color='blue'
            PolyLine(points, color=color, weight=lweight, opacity=1, popup=popup).add_to(feature_group_t1)
        else:
            color='cadetblue'
            PolyLine(points, color=color, weight=lweight, opacity=1, popup=popup).add_to(feature_group_t3)
        
    feature_group.add_to(m)
    feature_group_1.add_to(m)
    feature_group_2.add_to(m)
    feature_group_3.add_to(m)
    feature_group_t1.add_to(m)
    feature_group_t2.add_to(m)
    feature_group_t3.add_to(m)

# Cluster Functions

In [8]:
device = '352252060173789'
deviceDir = setSubDir(savedir, device)

In [9]:
tripsfile = setFile(deviceDir, "trips.p")
gcfile = setFile(deviceDir, "gc.p")

trips = getFile(tripsfile)
gc    = getFile(gcfile)

In [64]:
def createNetworkFoliumMap(gc):
    coms  = gc.getClusterCoMs()
    lat0  = coms[0].mean()
    long0 = coms[1].mean()
    m = Map(location=[lat0, long0], zoom_start=10)
    return m

In [71]:
def addClustersToMap(gc, m):

    feature_group_1 = FeatureGroup(name="Driver Top 90%")
    feature_group_2 = FeatureGroup(name="Driver Top 75%")
    feature_group_3 = FeatureGroup(name="Driver All")

    weights = Series([cluster.getCounts() for cl,cluster in gc.getClusters().items()])
    alpha   = weights.quantile(0.9)
    beta    = weights.quantile(0.75)
    
    for cl,cluster in gc.getClusters().items():
        com    = cluster.getCoM()
        rad    = max([cluster.getRadius(), 10])
        counts = cluster.getCounts()
        weight = float(counts)

        if counts >= alpha:
            Marker(com, icon=Icon(color='blue', icon_color='white', icon="car", angle=0, prefix='fa'), popup="").add_to(feature_group_1)
            Circle(com, color='darkblue', radius=rad, fill=True, fill_color='darkblue', weight=weight, opacity=0).add_to(feature_group_1)
        elif counts >= beta:
            Marker(com, icon=Icon(color='orange', icon_color='white', icon="car", angle=0, prefix='fa'), popup="").add_to(feature_group_2)
            Circle(com, color='orange', radius=rad, fill=True, fill_color='orange', weight=weight, opacity=0).add_to(feature_group_2)
        else:
            Circle(com, color='purple', radius=rad, fill=True, fill_color='purple', weight=weight, opacity=0).add_to(feature_group_3)            
        #addClusterToMap(com, rad, feature_group)

    feature_group_1.add_to(m)
    feature_group_2.add_to(m)
    feature_group_3.add_to(m)
    return m

In [72]:
from math import tanh
tanh(106/35)

0.9953287754278858

In [73]:
m = createNetworkFoliumMap(gc)
m = addClustersToMap(gc, m)

In [75]:
m